<a href="https://colab.research.google.com/github/kishoretvk/AIML/blob/main/Inspire_Mistral_7B_v2_Fine_Tuning_DPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Fine tune mistral 7b v2

In [ ]:
!pip install -q datasets trl peft bitsandbytes sentencepiece wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 85.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 195.4/195.4 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 263.5/263.5 kB 30.0 MB/s eta 0:00:00
     ━━━━━━

# Fine-tuning Mistral 7b v2 with DPO



### Setup Runtime
For fine-tuning Llama, a GPU instance is essential. Follow the directions below:

1. Go to `Runtime` (located in the top menu bar).
2. Select `Change Runtime Type`.
3. Choose `T4 GPU` (or a comparable option).


### Package Installation

Before we get started, let's ensure we have all the necessary packages installed.

In [ ]:
import os
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer
import bitsandbytes as bnb
from google.colab import userdata
import wandb

# Defined in the secrets tab in Google Colab
hf_token = userdata.get('huggingface')
# wb_token = userdata.get('wandb')
# wandb.login(key=wb_token)

#model_name = "teknium/OpenHermes-2.5-Mistral-7B"
#new_model = "NeuralHermes-2.5-Mistral-7B"

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

new_model = "inspire-Mistral-7B-v2-DPO-Math"

# ADD VCF Data here

In [ ]:
import json

# Assuming 'dataset' is your existing dataset and 'chatml_format' is the function you've optimized

# Define the JSON object
visa_format_t1_record = {
    "system": "You are an AI assistant. You will be given a task. You must generate a detailed and long answer.",
    "question": "What are the columns of a Visa Commercial Format T1 record?",
    "chosen":  {
        "Record Type": "Indicates the type of record, usually identified as 'T1'.",
        "Transaction Amount": "The total amount of the transaction.",
        "Transaction Date": "The date when the transaction occurred.",
        "Merchant Category Code (MCC)": "A code that classifies the merchant's primary business.",
        "Merchant Name and Location": "Information about the merchant including name, address, and possibly location details.",
        "Merchant ID (MID)": "A unique identifier assigned to the merchant.",
        "Transaction Reference Number": "A unique identifier assigned to the transaction.",
        "Authorization Code": "A code provided by the issuer authorizing the transaction.",
        "Cardholder Name": "The name of the cardholder.",
        "Card Number": "The credit card number used for the transaction.",
        "Card Expiration Date": "The expiration date of the credit card.",
        "Transaction Currency Code": "The currency used for the transaction.",
        "Tax Amount": "The amount of tax applied to the transaction.",
        "Tax Identification Number (TIN)": "The tax identification number associated with the merchant.",
        "Tax Rate": "The tax rate applied to the transaction.",
        "Invoice Number": "The invoice number associated with the transaction.",
        "Order Number": "The order number associated with the transaction.",
        "Customer Code": "A code identifying the customer associated with the transaction.",
        "Additional Data": "Additional information related to the transaction, such as line item details or special instructions.",
        "Reserved Fields": "Fields reserved for future use or specific implementation requirements.",
        "system": "You are an AI assistant. You will be given a task. You must generate a detailed and long answer."
    },
    "rejected": "Detailed list of columns and specific rules, it's recommended to refer to Visa's official documentation or the administration settings in Visa Commercial Pay Online."
}
visa_format_t2_record = {
  "question": "What are the specifications for transaction data?",
  "chosen": {
    "columns_specifications": {
      "Transaction ID": "A unique identifier assigned to each transaction.",
      "Transaction Date and Time": "The date and time when the transaction occurred, typically in UTC or a specified timezone.",
      "Transaction Amount": "The amount of the transaction, usually in the currency of the transaction.",
      "Merchant ID": "An identifier assigned to the merchant by the payment processor or acquirer.",
      "Merchant Name": "The name of the merchant where the transaction took place.",
      "Merchant Location": "The physical location of the merchant, including city, state, and country.",
      "Cardholder Name": "The name of the cardholder associated with the payment card used in the transaction.",
      "Cardholder Account Number": "The masked or truncated account number of the payment card used in the transaction, for security reasons.",
      "Card Type": "The type of payment card used in the transaction (e.g., Visa, Mastercard, American Express).",
      "Authorization Code": "A code provided by the issuer authorizing the transaction.",
      "Transaction Status": "The status of the transaction (e.g., approved, declined, pending).",
      "Transaction Type": "The type of transaction (e.g., purchase, refund, authorization, void).",
      "Currency Code": "The code representing the currency used in the transaction (e.g., USD, EUR, GBP).",
      "Payment Gateway/Processor": "The name or identifier of the payment gateway or processor involved in processing the transaction.",
      "Settlement Date": "The date when the transaction is settled and funds are transferred to the merchant's account.",
      "Fee Details": "Any fees associated with the transaction, including interchange fees, processing fees, and other charges."
    },
    "rejected": "Detailed list of columns and specific rules, it's recommended to refer to Visa's official documentation or the administration settings in Visa Commercial Pay Online."
  }
}

visa_format_t3_record = {
  "question": "What are the specifications for a T3 record in Visa Commercial Format?",
  "chosen": {
    "Transaction Identifier": "A unique number identifying the transaction.",
    "Cardholder Name": "The name of the individual who made the transaction.",
    "Merchant Name": "The name of the merchant where the transaction took place.",
    "Transaction Amount": "The amount of the transaction.",
    "Date": "The date when the transaction occurred.",
    "Merchant Category Code (MCC)": "A code that categorizes the merchant’s type of business.",
    "Original Currency Code": "The currency code of the original transaction amount.",
    "Settlement Amount": "The settled transaction amount in the billing currency.",
    "Approval Code": "A code indicating that the transaction was approved.",
    "Transaction Type": "The kind of transaction made.",
    "Source Currency Code": "The currency code of the country where the transaction was initiated.",
    "Destination Currency Code": "The currency code to which the transaction amount was converted, if applicable.",
    "Exchange Rate": "The rate at which the original currency was converted to the destination currency.",
    "Tax Amount": "The amount of tax applied to the transaction.",
    "Source Amount": "The transaction amount in the source currency.",
    "Destination Amount": "The transaction amount in the destination currency.",
    "Card Number": "The number of the card used for the transaction.",
    "Transaction Description": "A description of the transaction.",
    "Merchant City": "The city where the merchant is located.",
    "Merchant State/Province": "The state or province where the merchant is located.",
    "Merchant Postal Code": "The postal code of the merchant’s location.",
    "Merchant Country": "The country where the merchant is located.",
    "Tax ID": "The tax identification number associated with the transaction.",
    "Reference Number": "A number that references the transaction.",
    "Authorization Code": "The code received upon authorization of the transaction.",
    "Point of Sale (POS) Entry Mode": "The method by which the card information was entered.",
    "Service Code": "A code representing the card service used.",
    "Product ID": "An identifier for the product involved in the transaction.",
    "Quantity": "The quantity of the product purchased.",
    "Unit of Measure": "The unit in which the product quantity is measured.",
    "Unit Cost": "The cost per unit of the product.",
    "Total Cost": "The total cost of the product purchased.",
    "Discount Amount": "Any discount applied to the transaction.",
    "Net Amount": "The net amount after discounts.",
    "Freight Amount": "The cost of freight if applicable.",
    "Duty Amount": "Any duty applied to the transaction.",
    "Destination Country Code": "The country code of the destination.",
    "Ship-to ZIP/Postal Code": "The postal code to which the product was shipped.",
    "Rate of Exchange": "The exchange rate applied to the transaction.",
    "Goods and Services Tax (GST) Amount": "The amount of GST applied.",
    "Provincial Sales Tax (PST) Amount": "The amount of PST applied.",
    "Harmonized Sales Tax (HST) Amount": "The amount of HST applied.",
    "Miscellaneous Taxes": "Any other taxes applied to the transaction.",
    "Order Number": "The number associated with the order.",
    "Item Descriptor": "A description of the item.",
    "Product Code": "A code associated with the product.",
    "Item Commodity Code": "A code categorizing the commodity.",
    "Ship-from ZIP/Postal Code": "The postal code from where the item was shipped.",
    "Extended Item Amount": "The extended amount for the item.",
    "Discount Indicator": "An indicator of whether a discount was applied.",
    "Debit/Credit Indicator": "Indicates whether the transaction was a debit or credit.",
    "Replacement Amount": "The amount that replaces the original transaction amount in case of corrections.",
    "Currency Conversion Indicator": "Indicates whether currency conversion was involved.",
    "Card Acceptor ID": "An identifier for the merchant accepting the card.",
    "Terminal ID": "The identifier for the terminal where the transaction was processed.",
    "Authorization Response Code": "The response code from the authorization process.",
    "Transaction Security Code": "A code related to the security of the transaction.",
    "Risk Management Approval Status": "The status of the transaction’s risk management approval."
  },
  "rejected": "For detailed information on each column and how to customize your reports, you may refer to the Visa Commercial Pay Reporting documentation. Please note that the actual columns included in a T3 record can vary depending on the specific needs and configurations set by the Visa Commercial Pay administrators."
}


visa_format_t4_record ={
  "question": "What are the specifications for a T4 record in the Visa Commercial Format?",
  "chosen": {
    "columns_specifications": {
      "Batch Identifier": "A unique identifier for the batch of transactions.",
      "Batch Date": "The processing date for the batch.",
      "Total Transaction Amount": "Aggregate amount of all transactions in the batch.",
      "Total Settlement Amount": "Total amount settled in the batch’s currency.",
      "Currency Code": "The code for the currency used in the batch.",
      "Number of Transactions": "Count of all transactions within the batch.",
      "Credit Total": "Sum of amounts from credit transactions.",
      "Debit Total": "Sum of amounts from debit transactions.",
      "Return Total": "Total amount of returned transactions.",
      "Adjustment Total": "Sum of adjustments made to transactions in the batch.",
      "Discount Total": "Total of all discounts applied within the batch.",
      "Service Fee Total": "Sum of all service fees from the batch.",
      "Interchange Fee Total": "Total interchange fees for the batch.",
      "Batch Close Time": "The exact time the batch was closed.",
      "Merchant Category Code Total": "Aggregated amount for each MCC in the batch.",
      "Point of Sale Entry Mode Total": "Total amount for each POS entry mode."
    },
    "additional_columns": {
      "Smart Description": "A composite field showing key transaction data in one column, similar to a transaction narrative but with more detail.",
      "Custom Data Fields": "Administrators can add custom data fields to ensure data consistency for every deployment."
    },
    "reference": "For the most accurate and comprehensive details, including any additional columns and their specific configurations, please refer to the official Visa documentation or contact Visa directly. The Visa Commercial Format is subject to change, and the actual columns can vary depending on the specific needs and configurations set by the Visa Commercial Pay administrators."
  },
  "rejected": "Not applicable for this record."
}
visa_format_t5_record = {
  "question": "What are the specifications for a T5 record in the Visa Commercial Format?",
  "chosen": {
    "columns_specifications": {
      "Record Type": "Indicates the type of record, which would be T5 in this case.",
      "Report Name": "The name of the report this record is associated with.",
      "Creation Date": "The date the report was generated.",
      "Creator Identifier": "An identifier for the entity that created the report.",
      "Report Version": "The version of the report format.",
      "Column Count": "The number of columns in the report.",
      "Row Count": "The number of rows in the report.",
      "Data Start Row": "The row number where the data begins in the report.",
      "Data End Row": "The row number where the data ends in the report.",
      "Report Start Date": "The start date for the data included in the report.",
      "Report End Date": "The end date for the data included in the report."
    },
    "additional_columns": {
      "Custom Data Fields": "Administrators can add custom data fields to ensure data consistency for every deployment.",
      "Smart Description": "A composite field showing key transaction data in one column, similar to a transaction narrative but with more detail."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_t6_record = {
  "question": "What are the specifications for a T6 record in the Visa Commercial Format?",
  "chosen": {
    "columns_specifications": {
      "Record Type": "Indicates the type of record, which would be T6 in this case.",
      "Report Name": "The name of the report this record is associated with.",
      "Creation Date": "The date the report was generated.",
      "Creator Identifier": "An identifier for the entity that created the report.",
      "Report Version": "The version of the report format.",
      "Column Count": "The number of columns in the report.",
      "Row Count": "The number of rows in the report.",
      "Data Start Row": "The row number where the data begins in the report.",
      "Data End Row": "The row number where the data ends in the report.",
      "Report Start Date": "The start date for the data included in the report.",
      "Report End Date": "The end date for the data included in the report."
    },
    "additional_columns": {
      "Custom Data Fields": "Administrators can add custom data fields to ensure data consistency for every deployment.",
      "Smart Description": "A composite field showing key transaction data in one column, similar to a transaction narrative but with more detail."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_t7_record ={
  "question": "What are the specifications for a T7 record in the Visa Commercial Format?",
  "chosen": {
    "columns_specifications": {
      "Record Type": "Indicates the type of record, which would be T7 in this case.",
      "Report Name": "The name of the report this record is associated with.",
      "Creation Date": "The date the report was generated.",
      "Creator Identifier": "An identifier for the entity that created the report.",
      "Report Version": "The version of the report format.",
      "Column Count": "The number of columns in the report.",
      "Row Count": "The number of rows in the report.",
      "Data Start Row": "The row number where the data begins in the report.",
      "Data End Row": "The row number where the data ends in the report.",
      "Report Start Date": "The start date for the data included in the report.",
      "Report End Date": "The end date for the data included in the report."
    },
    "additional_columns": {
      "Custom Data Fields": "Administrators can add custom data fields to ensure data consistency for every deployment.",
      "Smart Description": "A composite field showing key transaction data in one column, similar to a transaction narrative but with more detail."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_t8_record ={
  "question": "What are the standard columns for a T8 record in the Visa Commercial Format?",
  "chosen": {
    "columns_specifications": {
      "Record Type": "Indicates the type of record, which would be T8 in this case.",
      "Custom Data Fields": "Templates can be created for custom data fields to ensure data consistency across deployments. This allows for the inclusion of data specific to the organization’s needs.",
      "Smart Description": "A composite field that can display key transaction data in one column, providing a detailed narrative of the transaction.",
      "Linked Card Pool Groups": "Displays any other card pool groups that the current card pool is linked to, allowing for integrated reporting across multiple card pools.",
      "Linked App Deployments": "Shows a list of app deployments for users who have access to the Visa Commercial Pay App, which can be useful for tracking and reporting purposes.",
      "Manage Mandatory Data": "This feature allows certain data fields in the card deployment process to be configured as mandatory, ensuring that crucial information is always captured.",
      "Payment Restriction Options": "During the deployment process, users can be prompted to select appropriate payment restrictions from a pre-configured dropdown, which can then be reported on in the T8 record."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_t9_record = {
  "question": "What are the specifications for a T9 record in the Visa Commercial Format?",
  "chosen": {
    "columns_specifications": {
      "Record Type": "Indicates the type of record, which would be T9 in this case.",
      "Report Name": "The name of the report this record is associated with.",
      "Creation Date": "The date the report was generated.",
      "Creator Identifier": "An identifier for the entity that created the report.",
      "Report Version": "The version of the report format.",
      "Column Count": "The number of columns in the report.",
      "Row Count": "The number of rows in the report.",
      "Data Start Row": "The row number where the data begins in the report.",
      "Data End Row": "The row number where the data ends in the report.",
      "Report Start Date": "The start date for the data included in the report.",
      "Report End Date": "The end date for the data included in the report."
    },
    "additional_columns": {
      "Custom Data Fields": "Administrators can add custom data fields to ensure data consistency for every deployment.",
      "Smart Description": "A composite field showing key transaction data in one column, providing a detailed narrative of the transaction.",
      "Linked Card Pool Groups": "Displays any other card pool groups that the current card pool is linked to, allowing for integrated reporting across multiple card pools.",
      "Linked App Deployments": "Shows a list of app deployments for users who have access to the Visa Commercial Pay App, which can be useful for tracking and reporting purposes.",
      "Manage Mandatory Data": "This feature allows certain data fields in the card deployment process to be configured as mandatory, ensuring that crucial information is always captured.",
      "Payment Restriction Options": "During the deployment process, users can be prompted to select appropriate payment restrictions from a pre-configured dropdown, which can then be reported on in the T9 record."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_t10_record = {
  "question": "What are the specifications for a T10 record in the Visa Commercial Format?",
  "chosen": {
    "columns_specifications": {
      "Record Type": "Indicates the type of record, which would be T10 in this case.",
      "Report Name": "The name of the report this record is associated with.",
      "Creation Date": "The date the report was generated.",
      "Creator Identifier": "An identifier for the entity that created the report.",
      "Report Version": "The version of the report format.",
      "Column Count": "The number of columns in the report.",
      "Row Count": "The number of rows in the report.",
      "Data Start Row": "The row number where the data begins in the report.",
      "Data End Row": "The row number where the data ends in the report.",
      "Report Start Date": "The start date for the data included in the report.",
      "Report End Date": "The end date for the data included in the report."
    },
    "additional_columns": {
      "Custom Data Fields": "Administrators can add custom data fields to ensure data consistency for every deployment.",
      "Smart Description": "A composite field showing key transaction data in one column, providing a detailed narrative of the transaction.",
      "Linked Card Pool Groups": "Displays any other card pool groups that the current card pool is linked to, allowing for integrated reporting across multiple card pools.",
      "Linked App Deployments": "Shows a list of app deployments for users who have access to the Visa Commercial Pay App, which can be useful for tracking and reporting purposes.",
      "Manage Mandatory Data": "This feature allows certain data fields in the card deployment process to be configured as mandatory, ensuring that crucial information is always captured.",
      "Payment Restriction Options": "During the deployment process, users can be prompted to select appropriate payment restrictions from a pre-configured dropdown, which can then be reported on in the T10 record."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_t11_record = {
  "question": "What are the potential specifications for a T11 record in the Visa Commercial Format?",
  "chosen": {
    "columns_specifications": {
      "Record Type": "Indicates the type of record, which would be T11 in this case.",
      "Report Name": "The name of the report this record is associated with.",
      "Creation Date": "The date the report was generated.",
      "Creator Identifier": "An identifier for the entity that created the report.",
      "Report Version": "The version of the report format.",
      "Column Count": "The number of columns in the report.",
      "Row Count": "The number of rows in the report.",
      "Data Start Row": "The row number where the data begins in the report.",
      "Data End Row": "The row number where the data ends in the report.",
      "Report Start Date": "The start date for the data included in the report.",
      "Report End Date": "The end date for the data included in the report."
    },
    "additional_columns": {
      "Custom Data Fields": "Administrators can add custom data fields to ensure data consistency for every deployment.",
      "Smart Description": "A composite field showing key transaction data in one column, providing a detailed narrative of the transaction.",
      "Linked Card Pool Groups": "Displays any other card pool groups that the current card pool is linked to, allowing for integrated reporting across multiple card pools.",
      "Linked App Deployments": "Shows a list of app deployments for users who have access to the Visa Commercial Pay App, which can be useful for tracking and reporting purposes.",
      "Manage Mandatory Data": "This feature allows certain data fields in the card deployment process to be configured as mandatory, ensuring that crucial information is always captured.",
      "Payment Restriction Options": "During the deployment process, users can be prompted to select appropriate payment restrictions from a pre-configured dropdown, which can then be reported on in the T11 record."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_t12_record = {
  "question": "What are the specifications for a T12 record in the Visa Commercial Format?",
  "chosen": {
    "columns_specifications": {
      "Record Type": "Indicates the type of record, which would be T12 in this case.",
      "Report Name": "The name of the report this record is associated with.",
      "Creation Date": "The date the report was generated.",
      "Creator Identifier": "An identifier for the entity that created the report.",
      "Report Version": "The version of the report format.",
      "Column Count": "The number of columns in the report.",
      "Row Count": "The number of rows in the report.",
      "Data Start Row": "The row number where the data begins in the report.",
      "Data End Row": "The row number where the data ends in the report.",
      "Report Start Date": "The start date for the data included in the report.",
      "Report End Date": "The end date for the data included in the report."
    },
    "additional_columns": {
      "Custom Data Fields": "Administrators can add custom data fields to ensure data consistency for every deployment.",
      "Smart Description": "A composite field showing key transaction data in one column, providing a detailed narrative of the transaction.",
      "Linked Card Pool Groups": "Displays any other card pool groups that the current card pool is linked to, allowing for integrated reporting across multiple card pools.",
      "Linked App Deployments": "Shows a list of app deployments for users who have access to the Visa Commercial Pay App, which can be useful for tracking and reporting purposes.",
      "Manage Mandatory Data": "This feature allows certain data fields in the card deployment process to be configured as mandatory, ensuring that crucial information is always captured.",
      "Payment Restriction Options": "During the deployment process, users can be prompted to select appropriate payment restrictions from a pre-configured dropdown, which can then be reported on in the T12 record."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_t13_record = {
  "question": "What are the potential specifications for a T13 record in the Visa Commercial Format?",
  "chosen": {
    "columns_specifications": {
      "Record Type": "Indicates the type of record, which would be T13 in this case.",
      "Report Name": "The name of the report this record is associated with.",
      "Creation Date": "The date the report was generated.",
      "Creator Identifier": "An identifier for the entity that created the report.",
      "Report Version": "The version of the report format.",
      "Column Count": "The number of columns in the report.",
      "Row Count": "The number of rows in the report.",
      "Data Start Row": "The row number where the data begins in the report.",
      "Data End Row": "The row number where the data ends in the report.",
      "Report Start Date": "The start date for the data included in the report.",
      "Report End Date": "The end date for the data included in the report."
    },
    "additional_columns": {
      "Custom Data Fields": "Administrators can add custom data fields to ensure data consistency for every deployment.",
      "Smart Description": "A composite field showing key transaction data in one column, providing a detailed narrative of the transaction.",
      "Linked Card Pool Groups": "Displays any other card pool groups that the current card pool is linked to, allowing for integrated reporting across multiple card pools.",
      "Linked App Deployments": "Shows a list of app deployments for users who have access to the Visa Commercial Pay App, which can be useful for tracking and reporting purposes.",
      "Manage Mandatory Data": "This feature allows certain data fields in the card deployment process to be configured as mandatory, ensuring that crucial information is always captured.",
      "Payment Restriction Options": "During the deployment process, users can be prompted to select appropriate payment restrictions from a pre-configured dropdown, which can then be reported on in the T13 record."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_t14_record = {
  "question": "What are the potential specifications for a T14 record in the Visa Commercial Format?",
  "chosen": {
    "columns_specifications": {
      "Record Type": "The type of record, T14 in this case.",
      "Report Name": "The name of the report associated with this record.",
      "Creation Date": "The date when the report was generated.",
      "Creator Identifier": "An identifier for the creator of the report.",
      "Report Version": "The version number of the report format.",
      "Column Count": "The total number of columns in the report.",
      "Row Count": "The total number of rows in the report.",
      "Data Start Row": "The row number where the data starts.",
      "Data End Row": "The row number where the data ends.",
      "Report Start Date": "The starting date for the data included in the report.",
      "Report End Date": "The ending date for the data included in the report."
    },
    "additional_columns": {
      "Custom Data Fields": "Fields added by administrators to ensure data consistency across reports.",
      "Smart Description": "A composite field showing detailed transaction data in one column.",
      "Linked Card Pool Groups": "Information on any card pool groups linked to the current report.",
      "Linked App Deployments": "A list of app deployments associated with the users who have access to the Visa Commercial Pay App.",
      "Manage Mandatory Data": "Settings that make certain data fields mandatory during the card deployment process.",
      "Payment Restriction Options": "Options for payment restrictions that can be configured during the deployment process."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_t15_record = {
  "question": "What are the potential specifications for a T15 record in the Visa Commercial Format?",
  "chosen": {
    "columns_specifications": {
      "Record Type": "Identifies the T15 record type.",
      "Report Name": "The name of the associated report.",
      "Creation Date": "When the report was generated.",
      "Creator Identifier": "Identifier for the report creator.",
      "Report Version": "Version of the report format.",
      "Column Count": "Number of columns in the report.",
      "Row Count": "Number of rows in the report.",
      "Data Start Row": "Where report data starts.",
      "Data End Row": "Where report data ends.",
      "Report Start Date": "Start date for the report data.",
      "Report End Date": "End date for the report data."
    },
    "additional_columns": {
      "Custom Data Fields": "Fields added by administrators for data consistency.",
      "Smart Description": "Detailed transaction data in one column.",
      "Linked Card Pool Groups": "Information on linked card pool groups for integrated reporting.",
      "Linked App Deployments": "List of app deployments for users with access to the Visa Commercial Pay App.",
      "Manage Mandatory Data": "Settings to make certain data fields mandatory during card deployment.",
      "Payment Restriction Options": "Configurable payment restrictions during the deployment process."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_t16_record = {
  "question": "What are the potential specifications for a T16 record in the Visa Commercial Format?",
  "chosen": {
    "columns_specifications": {
      "Record Type": "Identifies the T16 record type.",
      "Report Name": "The name of the associated report.",
      "Creation Date": "When the report was generated.",
      "Creator Identifier": "Identifier for the report creator.",
      "Report Version": "Version of the report format.",
      "Column Count": "Number of columns in the report.",
      "Row Count": "Number of rows in the report.",
      "Data Start Row": "Where report data starts.",
      "Data End Row": "Where report data ends.",
      "Report Start Date": "Start date for the report data.",
      "Report End Date": "End date for the report data."
    },
    "additional_columns": {
      "Custom Data Fields": "Fields added by administrators for data consistency.",
      "Smart Description": "Detailed transaction data in one column.",
      "Linked Card Pool Groups": "Information on linked card pool groups for integrated reporting.",
      "Linked App Deployments": "List of app deployments for users with access to the Visa Commercial Pay App.",
      "Manage Mandatory Data": "Settings to make certain data fields mandatory during card deployment.",
      "Payment Restriction Options": "Configurable payment restrictions during the deployment process."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_Account_Balance_record = {
  "question": "What are the typical attributes included in the Account Balance record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique number identifying the commercial account.",
      "Balance Date": "The date when the balance is reported.",
      "Available Credit": "The amount of credit available at the time of the report.",
      "Current Balance": "The total current balance on the account.",
      "Pending Charges": "Charges that have been authorized but not yet posted to the account.",
      "Currency Code": "The currency in which the account is maintained."
    },
    "optional_attributes": {
      "Credit Limit": "The maximum amount of credit available on the account.",
      "Past Due Amount": "Any amount on the account that is past the payment due date.",
      "Payment Due Date": "The date by which the next payment should be made."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_Allocation_record  = {
  "question": "What are the typical attributes included in the Allocation record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique number identifying the commercial account.",
      "Transaction Date": "The date when the transaction occurred.",
      "Merchant Name": "The name of the merchant where the transaction took place.",
      "Total Amount": "The total cost of the transaction.",
      "Allocation Amount": "The portion of the total amount allocated to a specific cost center.",
      "Cost Center": "The department or unit within the company to which the transaction amount is allocated."
    },
    "optional_attributes": {
      "Project Code": "A code associated with a specific project for tracking expenses.",
      "Business Unit": "The specific business unit within the company that is responsible for the transaction.",
      "Expense Type": "The category of the expense, such as travel, supplies, or services."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_Allocation_Description_record  = {
  "question": "What are the typical attributes included in the Allocation Description record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Description": "A detailed explanation of the allocation.",
      "Reference": "Any reference numbers or codes associated with the allocation.",
      "Amount": "The specific amount of money allocated.",
      "Date": "The date of the allocation."
    },
    "optional_attributes": {
      "Project Name": "The name of the project to which the allocation is related.",
      "Business Justification": "The reason for the allocation.",
      "Department Code": "The code for the department receiving the allocation."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_Temporary_Services_record = {
  "question": "What are the typical attributes included in the Temporary Services record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique number identifying the commercial account.",
      "Transaction Date": "The date when the transaction occurred.",
      "Merchant Name": "The name of the service provider.",
      "Total Amount": "The total cost of the services provided.",
      "Service Description": "A description of the services rendered.",
      "Start Date": "The start date of the service period.",
      "End Date": "The end date of the service period."
    },
    "optional_attributes": {
      "Project Code": "A code associated with a specific project for tracking expenses.",
      "Hours Worked": "The number of hours worked if the service is billed hourly.",
      "Rate": "The rate charged per hour or per service."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_Shipping_Services_record  = {
  "question": "What are the typical attributes included in the Shipping Services record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique number identifying the commercial account.",
      "Transaction Date": "The date when the transaction occurred.",
      "Merchant Name": "The name of the shipping service provider.",
      "Total Shipping Cost": "The total cost of the shipping service.",
      "Service Level": "The level of service provided (e.g., standard, express).",
      "Tracking Number": "The tracking number associated with the shipment.",
      "Weight": "The weight of the shipped item(s).",
      "Origin": "The starting location of the shipment.",
      "Destination": "The final destination of the shipment."
    },
    "optional_attributes": {
      "Reference Number": "A number used to identify the transaction in the company’s system.",
      "Item Description": "A description of the item(s) being shipped.",
      "Insurance Amount": "The amount of insurance coverage for the shipment."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_car_rental_detail_record  = {
  "question": "What are the typical attributes included in the Car Rental Detail record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique number identifying the commercial account.",
      "Rental Agreement Number": "The number associated with the car rental agreement.",
      "Rental Period": "The start and end dates of the car rental.",
      "Daily Rental Rate": "The cost per day of the car rental.",
      "Total Rental Cost": "The total cost of the rental period.",
      "Vehicle Class": "The class or type of vehicle rented.",
      "Mileage": "The number of miles driven during the rental period."
    },
    "optional_attributes": {
      "Fuel Charges": "Charges for fuel if not returned with a full tank.",
      "Insurance Charges": "Charges for any additional insurance taken.",
      "Late Return Fee": "Fees applied if the vehicle is returned after the agreed-upon time."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_phone_record = {
  "question": "What are the typical attributes included in the Phone record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique number identifying the commercial account.",
      "Transaction Date": "The date when the transaction occurred.",
      "Merchant Name": "The name of the telecommunication service provider.",
      "Total Amount": "The total cost of the phone services.",
      "Service Period": "The billing period for the services.",
      "Call Detail": "Information about individual calls, such as date, time, duration, and cost."
    },
    "optional_attributes": {
      "International Charges": "Charges for international calls if applicable.",
      "Roaming Charges": "Charges incurred for phone usage outside the service area.",
      "Data Usage": "The amount of data used during the billing period."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_car_rental_summary_record  = {
  "question": "What are the typical attributes included in the Car Rental Summary record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique number identifying the commercial account.",
      "Total Rental Days": "The total number of days the vehicle was rented.",
      "Total Rental Cost": "The aggregate cost of all rentals during the reporting period.",
      "Number of Rentals": "The total number of rental transactions.",
      "Average Daily Rate": "The average cost per day across all rentals."
    },
    "optional_attributes": {
      "Late Return Fees": "Total fees charged for late returns.",
      "Fuel Charges": "Total charges for fuel if applicable.",
      "Insurance Charges": "Total charges for any additional insurance taken."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_lodging_summary_record  = {
  "question": "What are the typical attributes included in the Lodging Summary record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Total Lodging Expenses": "The total amount spent on lodging.",
      "Number of Stays": "The total number of lodging transactions.",
      "Average Cost per Stay": "The average cost of each stay.",
      "Total Nights": "The total number of nights across all stays.",
      "Merchant Category Code (MCC)": "The code that identifies the type of lodging services."
    },
    "optional_attributes": {
      "City and Country of Lodging": "The location where the lodging services were used.",
      "Date Range": "The period over which the lodging expenses were incurred.",
      "Business Purpose": "A brief description of the business purpose for the lodging expenses."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_line_item_detail_record  = {
  "question": "What are the typical attributes included in the Line Item Detail record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique number identifying the commercial account.",
      "Transaction Date": "The date when the transaction occurred.",
      "Merchant Name": "The name of the merchant where the transaction took place.",
      "Item Description": "A description of the item purchased.",
      "Quantity": "The number of items purchased.",
      "Unit Cost": "The cost per unit of the item.",
      "Total Cost": "The total cost for the item(s)."
    },
    "optional_attributes": {
      "Product Code": "A code that identifies the specific item.",
      "Item Category": "The category to which the item belongs.",
      "Tax Amount": "The amount of tax applied to the item."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_line_item_summary_record  = {
  "question": "What are the typical attributes included in the Line Item Summary record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique number identifying the commercial account.",
      "Total Number of Line Items": "The total count of all line items within the transaction.",
      "Total Amount": "The aggregate amount of all line items.",
      "Average Item Cost": "The average cost per line item."
    },
    "optional_attributes": {
      "Total Tax Amount": "The total amount of tax for all line items.",
      "Total Shipping Cost": "The total cost of shipping for the line items.",
      "Merchant Category Code (MCC)": "The code that identifies the type of goods or services provided."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_passenger_itinerary_record  = {
  "question": "What are the typical attributes included in the Passenger Itinerary record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique number identifying the commercial account.",
      "Itinerary Number": "A number that uniquely identifies the travel itinerary.",
      "Traveler Name": "The name of the individual traveling.",
      "Departure and Arrival Locations": "The starting point and destination of the travel.",
      "Dates of Travel": "The start and end dates of the trip.",
      "Airline or Rail Service Provider": "The company providing the travel service.",
      "Ticket Number": "The number associated with the travel ticket.",
      "Fare": "The cost of the travel."
    },
    "optional_attributes": {
      "Class of Service": "The class in which the individual is traveling (e.g., economy, business).",
      "Seat Number": "The assigned seat for the traveler.",
      "Frequent Flyer Information": "Details of the traveler’s frequent flyer program."
    }
  },
  "rejected": "Not applicable for this record."
}
visa_format_leg_specific_information_record  = {
"question": "What are the typical attributes included in the Passenger Itinerary record in the Visa Commercial Format?",
"chosen": {
"attributes": {
"Account Number": "The unique number identifying the commercial account.",
"Itinerary Number": "A number that uniquely identifies the travel itinerary.",
"Traveler Name": "The name of the individual traveling.",
"Departure and Arrival Locations": "The starting point and destination of the travel.",
"Dates of Travel": "The start and end dates of the trip.",
"Airline or Rail Service Provider": "The company providing the travel service.",
"Ticket Number": "The number associated with the travel ticket.",
"Fare": "The cost of the travel."
},
"optional_attributes": {
"Class of Service": "The class in which the individual is traveling (e.g., economy, business).",
"Seat Number": "The assigned seat for the traveler.",
"Frequent Flyer Information": "Details of the traveler’s frequent flyer program."
}
},
"rejected": "Not applicable for this record."
}
visa_format_fleet_service_record  = {
  "question": "What are the typical attributes included in the Passenger Itinerary record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique number identifying the commercial account.",
      "Itinerary Number": "A number that uniquely identifies the travel itinerary.",
      "Traveler Name": "The name of the individual traveling.",
      "Departure and Arrival Locations": "The starting point and destination of the travel.",
      "Dates of Travel": "The start and end dates of the trip.",
      "Airline or Rail Service Provider": "The company providing the travel service.",
      "Ticket Number": "The number associated with the travel ticket.",
      "Fare": "The cost of the travel."
    },
    "optional_attributes": {
      "Class of Service": "The class in which the individual is traveling (e.g., economy, business).",
      "Seat Number": "The assigned seat for the traveler.",
      "Frequent Flyer Information": "Details of the traveler’s frequent flyer program."
    }
  },
  "rejected": "For the specific specifications and attributes of the Passenger Itinerary record, you would need to refer to the official Visa documentation, such as the Visa Merchant Data Standards Manual1, or contact Visa directly, as this information is proprietary and not publicly available. If you have access to Visa’s technical documentation, it will include all the necessary details for the Passenger Itinerary record and any additional or optional fields it may contain."
}
visa_format_fleet_product_record  = {
  "question": "What are the typical attributes included in the Fleet Product record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique number identifying the commercial account.",
      "Product Code": "A code that identifies the specific product or service.",
      "Quantity": "The number of products or the extent of services provided.",
      "Unit Cost": "The cost per unit of the product or service.",
      "Total Cost": "The total cost for the product(s) or service(s).",
      "Transaction Date": "The date when the transaction occurred.",
      "Merchant Name": "The name of the merchant providing the fleet product or service."
    },
    "optional_attributes": {
      "Vehicle Identifier": "A unique identifier for the vehicle associated with the product or service.",
      "Service Category": "The category of service performed (e.g., maintenance, repair).",
      "Authorization Code": "A code indicating that the purchase was authorized by the company."
    }
  },
  "rejected": "For the specific specifications and attributes of the Fleet Product record, you would need to refer to the official Visa documentation, such as the Visa Merchant Data Standards Manual1, or contact Visa directly, as this information is proprietary and not publicly available. If you have access to Visa’s technical documentation, it will include all the necessary details for the Fleet Product record and any additional or optional fields it may contain."
}
visa_format_lodging_detail_record  = {
  "question": "What are the typical attributes included in the Lodging Detail record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique identifier for the commercial account.",
      "Merchant Name": "The name of the lodging establishment.",
      "Total Amount": "The total cost of the stay.",
      "Room Rate": "The cost per night.",
      "Number of Nights": "The duration of the stay.",
      "Room Tax": "Any taxes applied to the room rate.",
      "Incidentals": "Charges for additional services like room service."
    },
    "optional_attributes": {
      "Customer Code": "A code to identify the customer or company.",
      "Reservation ID": "A unique identifier for the reservation.",
      "Check-in/Check-out Time": "The times for check-in and check-out.",
      "No Show Indicator": "Whether the reservation was a no-show."
    }
  },
  "rejected": "For the specific specifications and attributes of the Lodging Detail record, you would need to refer to the official Visa documentation, such as the Visa Merchant Data Standards Manual1, or contact Visa directly, as this information is proprietary and not publicly available. If you have access to Visa’s technical documentation, it will include all the necessary details for the Lodging Detail record and any additional or optional fields it may contain."
}
visa_format_card_holder_optional_field_record  = {
  "question": "What are the typical attributes included in the Card Holder Optional Field in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Custom Data Fields": "These might include specific codes, identifiers, or descriptions that are unique to the cardholder’s company or the nature of the transaction.",
      "Control Types": "Settings that determine how the data can be entered or selected, ensuring consistency and accuracy.",
      "Default Values": "Pre-set values that can be automatically applied to transactions unless overridden by the user."
    },
    "optional_attributes": {
      "Data Lists": "Pre-defined lists from which users can select data, ensuring uniformity across transactions.",
      "Aliases": "Alternate names or labels for the same data field, which can help in aligning data from different sources or systems."
    }
  },
  "rejected": "For the specific specifications and attributes of the Card Holder Optional Field, you would need to refer to the official Visa documentation, such as the Visa Merchant Data Standards Manual1, or contact Visa directly, as this information is proprietary and not publicly available. If you have access to Visa’s technical documentation, it will include all the necessary details for the Card Holder Optional Field and any additional or optional fields it may contain."
}
visa_format_card_account_record  = {
  "question": "What are the typical attributes included in the Card Account record in the Visa Commercial Format?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique identifier for the card account.",
      "Cardholder Identification": "Information that links the account to a specific cardholder.",
      "Account Status": "The current status of the account (e.g., active, closed).",
      "Credit Limit": "The maximum amount of credit available on the account.",
      "Available Credit": "The amount of credit available at the time of the report.",
      "Currency Code": "The currency in which the account is maintained."
    },
    "optional_attributes": {
      "Payment Due Date": "The date by which the next payment should be made.",
      "Past Due Amount": "Any amount on the account that is past the payment due date.",
      "Balance": "The total current balance on the account."
    }
  },
  "rejected": "For the specific specifications and attributes of the Card Account record, you would need to refer to the official Visa documentation, such as the Visa Merchant Data Standards Manual1, or contact Visa directly, as this information is proprietary and not publicly available. If you have access to Visa’s technical documentation, it will include all the necessary details for the Card Account record and any additional or optional fields it may contain."
}
visa_format_card_transaction_record  = {
  "question": "What are the typical attributes included in the Card Transaction record in the Visa Commercial Pay system?",
  "chosen": {
    "attributes": {
      "Transaction ID": "The unique number assigned to each transaction when it is received from the issuer and imported into the system.",
      "Date": "The date that the merchant charged the Virtual Card Number (VCN).",
      "Category": "Based on the merchant category code.",
      "Narrative": "Usually the merchant’s name, referring to the text provided by the merchant when the transaction is processed.",
      "Card": "The last four digits of the VCN used to settle the transaction.",
      "Full Amount": "The amount charged to the card and the currency.",
      "Matched": "Indicates whether the transaction has been successfully matched to a deployment.",
      "Unreconciled Amount": "Shows the difference between the Full Amount and the total value of the invoice created. A green indicator means the transaction has been fully reconciled."
    },
    "optional_attributes": {
      "Deployment ID": "Identifies the specific deployment of funds.",
      "Spend Type": "The type of spend the card was created for.",
      "Creation Date": "The date the card was created.",
      "Supplier Name": "Name of the supplier as provided in the booking platform.",
      "Spend Date Range": "The date range for which the card was created.",
      "Matched Transactions": "The number of transactions matched to the deployment.",
      "Total Amount Deployed": "The total amount deployed at the time of booking.",
      "Merchant Category Code (MCC)": "The category based on the merchant’s classification.",
      "Custom Data Fields": "These fields can be chosen from a dropdown or free-typed for additional information.",
      "Statement Reference": "If a Card Pool is selected, a Statement Reference can be chosen from the dropdown list."
    }
  },
  "rejected": "For the specific specifications and attributes of the Card Account record, you would need to refer to the official Visa documentation, such as the Visa Merchant Data Standards Manual1, or contact Visa directly, as this information is proprietary and not publicly available. If you have access to Visa’s technical documentation, it will include all the necessary details for the Card Account record and any additional or optional fields it may contain."
}
visa_format_Account_record = {
  "question": "What are the typical attributes included in the Account Balance record in the Visa Commercial Format (VCF)?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique number identifying the account.",
      "Balance Date": "The date when the balance is reported.",
      "Currency Code": "The code for the currency in which the account is maintained.",
      "Ledger Balance": "The total balance in the account at the end of the business day.",
      "Available Balance": "The amount available for withdrawal or use, considering any holds or pending transactions.",
      "Pending Transactions": "Transactions that have been authorized but not yet posted to the account.",
      "Book Balance": "The balance in an account at the beginning of each day, plus or minus any day’s ledger transactions.",
      "Float Amount": "The amount of uncollected funds represented by checks in the process of clearing.",
      "Hold Amount": "The total amount of holds on the account, which could include card authorizations, check deposit holds, or other types of holds."
    },
    "optional_attributes": {}
  },
  "rejected": "Not applicable for this record."
}
visa_format_Temporary_Allocation_record = {
  "question": "What are the typical attributes included in the Allocation Description record in the Visa Commercial Format (VCF)?",
  "chosen": {
    "attributes": {
      "Account Number": "The unique number identifying the account involved in the allocation.",
      "Allocation Amount": "The amount of money allocated to a particular transaction or set of transactions.",
      "Allocation Date": "The date on which the allocation was made.",
      "Allocation ID": "A unique identifier for the allocation record.",
      "Business Unit": "The division or department within a company that the allocation is associated with.",
      "Cost Center": "The specific unit or location within the business unit that the allocation is charged to.",
      "General Ledger Code": "The code that corresponds to the general ledger account where the allocation is recorded.",
      "Project Code": "If the allocation is related to a specific project, this code identifies it.",
      "Expense Type": "The nature of the expense being allocated, such as travel, supplies, etc.",
      "Description": "A detailed description of the allocation, which may include information about the purpose or nature of the expense."
    },
    "optional_attributes": {}
  },
  "rejected": "Not applicable for this record."
}
visa_info = {
  "question": "Who are the key leaders at Visa Inc. as of March 2024?",
  "chosen": {
    "CEO": "Ryan McInerney, who has been serving as President since 2013, became the CEO effective February 1, 2023.",
    "Executive Chairman": "Alfred F. Kelly, Jr., the former CEO, transitioned to the role of Executive Chairman of the Board following McInerney’s appointment.",
    "Leadership Team": {
      "Antony Cahill": "Global Head of Value Added Services",
      "Frank Cooper III": "Chief Marketing Officer",
      "Paul D. Fabara": "Chief Risk Officer",
      "Jack Forestell": "Chief Product and Strategy Officer",
      "Michelle Gethers": "Chief Diversity Officer and Head of Corporate Responsibility",
      "Charlotte Hogg": "Chief Executive Officer, Europe",
      "Oliver Jenkyn": "Group President, Global Markets",
      "Christopher T. Newkirk": "Global Head of New Flows – Commercial & Money Movement Solutions",
      "Julie B. Rottenberg": "General Counsel",
      "Chris Suh": "Chief Financial Officer",
      "Rajat Taneja": "President, Technology",
      "Kelly Mahon Tullier": "Vice Chair, Chief People and Corporate Affairs Officer"
    }
  },
  "rejected": "For detailed and up-to-date information about Visa Inc.'s leadership, please refer to the company's official website or recent press releases."
}
# Convert the dictionary to a JSON string
visa_format_t1_record = json.dumps(visa_format_t1_record)
visa_format_t2_record = json.dumps(visa_format_t2_record)
visa_format_t3_record = json.dumps(visa_format_t3_record)
visa_format_t4_record = json.dumps(visa_format_t4_record)
visa_format_t5_record = json.dumps(visa_format_t5_record)
visa_format_t6_record = json.dumps(visa_format_t6_record)
visa_format_t7_record = json.dumps(visa_format_t7_record)
visa_format_t8_record = json.dumps(visa_format_t8_record)
visa_format_t9_record = json.dumps(visa_format_t9_record)
visa_format_t10_record = json.dumps(visa_format_t10_record)
visa_format_t11_record = json.dumps(visa_format_t11_record)
visa_format_t12_record = json.dumps(visa_format_t12_record)
visa_format_t13_record = json.dumps(visa_format_t13_record)
visa_format_t14_record = json.dumps(visa_format_t14_record)
visa_format_t15_record = json.dumps(visa_format_t15_record)
visa_format_t16_record = json.dumps(visa_format_t16_record)
visa_format_Account_record = json.dumps(visa_format_Account_record)
visa_format_Allocation_record = json.dumps(visa_format_Allocation_record)
visa_format_Allocation_Description_record = json.dumps(visa_format_Allocation_Description_record)
visa_format_Temporary_Allocation_record = json.dumps(visa_format_Temporary_Allocation_record)
#visa_format_Shipping_Services = json.dumps(visa_format_Shipping_Services)
#visa_format_Car_Rental_detail_record = json.dumps(visa_format_Car_Rental_detail_record)
visa_format_car_rental_detail_record = json.dumps(visa_format_car_rental_detail_record)
visa_format_phone_record = json.dumps(visa_format_phone_record)
visa_format_car_rental_summary_record = json.dumps(visa_format_car_rental_summary_record)
visa_format_lodging_summary_record  = json.dumps(visa_format_lodging_summary_record)
visa_format_line_item_detail_record = json.dumps(visa_format_line_item_detail_record)
visa_format_line_item_summary_record = json.dumps(visa_format_line_item_summary_record)
visa_format_passenger_itinerary_record = json.dumps(visa_format_passenger_itinerary_record)
visa_format_leg_specific_information_record = json.dumps(visa_format_leg_specific_information_record)
visa_format_fleet_service_record = json.dumps(visa_format_fleet_service_record)
visa_format_fleet_product_record = json.dumps(visa_format_fleet_product_record)
visa_format_lodging_detail_record = json.dumps(visa_format_lodging_detail_record)
visa_format_card_holder_optional_field_record = json.dumps(visa_format_card_holder_optional_field_record)
visa_format_card_account_record = json.dumps(visa_format_card_account_record)
visa_format_card_transaction_record = json.dumps(visa_format_card_transaction_record)
visa_info = json.dumps(visa_info)

# Print the JSON string
print(visa_format_t1_record)

{"system": "You are an AI assistant. You will be given a task. You must generate a detailed and long answer.", "question": "What are the columns of a Visa Commercial Format T1 record?", "chosen": {"Record Type": "Indicates the type of record, usually identified as 'T1'.", "Transaction Amount": "The total amount of the transaction.", "Transaction Date": "The date when the transaction occurred.", "Merchant Category Code (MCC)": "A code that classifies the merchant's primary business.", "Merchant Name and Location": "Information about the merchant including name, address, and possibly location details.", "Merchant ID (MID)": "A unique identifier assigned to the merchant.", "Transaction Reference Number": "A unique identifier assigned to the transaction.", "Authorization Code": "A code provided by the issuer authorizing the transaction.", "Cardholder Name": "The name of the cardholder.", "Card Number": "The credit card number used for the transaction.", "Card Expiration Date": "The expirat

In [ ]:
def chatml_format(example):
    # Initialize the formatted example dictionary
    formatted_example = {"prompt": "", "chosen": "", "rejected": ""}

    # Add system message if present
    system_content = example.get('system', '').strip()
    if system_content:
        formatted_example["prompt"] += f"system\n\n{system_content}\n\n"

    # Add user message
    user_content = example.get('question', '').strip()
    if user_content:
        formatted_example["prompt"] += f"user\n\n{user_content}\n\n"

    # Add chosen and rejected messages if present
    chosen_content = example.get('chosen', '').strip()
    if chosen_content:
        formatted_example["chosen"] = f"{chosen_content}\n"

    rejected_content = example.get('rejected', '').strip()
    if rejected_content:
        formatted_example["rejected"] = f"{rejected_content}\n"

    return formatted_example


# Load dataset
dataset = load_dataset("Intel/orca_dpo_pairs")['train']

print(dataset[1])
print("dataset type ")
print(type(dataset))
# Save columns
original_columns = dataset.column_names


# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "left"

# Format dataset
dataset = dataset.map(
    chatml_format,
    remove_columns=original_columns
)

# Print sample
dataset[1]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

{'system': 'You are an AI assistant. You will be given a task. You must generate a detailed and long answer.', 'question': 'Generate an approximately fifteen-word sentence that describes all this data: Midsummer House eatType restaurant; Midsummer House food Chinese; Midsummer House priceRange moderate; Midsummer House customer rating 3 out of 5; Midsummer House near All Bar One', 'chosen': 'Midsummer House is a moderately priced Chinese restaurant with a 3/5 customer rating, located near All Bar One.', 'rejected': ' Sure! Here\'s a sentence that describes all the data you provided:\n\n"Midsummer House is a moderately priced Chinese restaurant with a customer rating of 3 out of 5, located near All Bar One, offering a variety of delicious dishes."'}
dataset type 
<class 'datasets.arrow_dataset.Dataset'>


tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

Map:   0%|          | 0/12859 [00:00<?, ? examples/s]

{'chosen': 'Midsummer House is a moderately priced Chinese restaurant with a 3/5 customer rating, located near All Bar One.\n',
 'rejected': 'Sure! Here\'s a sentence that describes all the data you provided:\n\n"Midsummer House is a moderately priced Chinese restaurant with a customer rating of 3 out of 5, located near All Bar One, offering a variety of delicious dishes."\n',
 'prompt': 'system\n\nYou are an AI assistant. You will be given a task. You must generate a detailed and long answer.\n\nuser\n\nGenerate an approximately fifteen-word sentence that describes all this data: Midsummer House eatType restaurant; Midsummer House food Chinese; Midsummer House priceRange moderate; Midsummer House customer rating 3 out of 5; Midsummer House near All Bar One\n\n'}

In [ ]:

# # Concatenate the new Dataset with the existing one
# dataset = concatenate_datasets([dataset, formatted_entry_dataset])




#dataset = dataset.append(visa_format_t1_record)

# dataset = dataset.add_item(visa_format_t1_record)
# dataset.append(visa_format_t2_record)
# dataset.append(visa_format_t3_record)
# dataset.append(visa_format_t4_record)
# dataset.append(visa_format_t5_record)
# dataset.append(visa_format_t6_record)
# dataset.append(visa_format_t7_record)
# dataset.append(visa_format_t8_record)
# dataset.append(visa_format_t9_record)
# dataset.append(visa_format_t10_record)
# dataset.append(visa_format_t11_record)
# dataset.append(visa_format_t12_record)
# dataset.append(visa_format_t13_record)
# dataset.append(visa_format_t14_record)
# dataset.append(visa_format_t15_record)
# dataset.append(visa_format_t16_record)
# dataset.append(visa_format_Account_record)
# dataset.append(visa_format_Allocation_record)
# dataset.append(visa_format_Allocation_Description_record)
# dataset.append(visa_format_Temporary_Allocation_record)
# dataset.append(visa_format_car_rental_detail_record)
# dataset.append(visa_format_phone_record)
# dataset.append(visa_format_car_rental_summary_record)
# dataset.append(visa_format_lodging_summary_record)
# dataset.append(visa_format_line_item_detail_record)
# dataset.append(visa_format_line_item_summary_record)
# dataset.append(visa_format_passenger_itinerary_record)
# dataset.append(visa_format_leg_specific_information_record)
# dataset.append(visa_format_fleet_service_record)
# dataset.append(visa_format_fleet_product_record)
# dataset.append(visa_format_lodging_detail_record)
# dataset.append(visa_format_card_holder_optional_field_record)
# dataset.append(visa_format_card_account_record)
# dataset.append(visa_format_card_transaction_record)
# dataset.append(visa_info)


###  Train with DPO


In [ ]:
!pip install accelerate
!pip install -U bitsandbytes
from transformers import AutoModelForCausalLM, BitsAndBytesConfig


# Create a BitsAndBytesConfig object with the desired settings
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    # Add any other configurations you need here
)

In [ ]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113


In [ ]:
import torch
import gc

# Before starting your training, clear the cache and collect garbage
torch.cuda.empty_cache()
gc.collect()

# Modify your training arguments to reduce the batch size
# Modify your training arguments to reduce the batch size
training_args = TrainingArguments(
    output_dir='./results',  # The output directory where the model predictions and checkpoints will be written
    per_device_train_batch_size=2,  # Reduced batch size to save memory
    gradient_accumulation_steps=8,  # Increased gradient accumulation to maintain effective batch size
    num_train_epochs=3,  # Total number of training epochs to perform
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=10,  # Log every X updates steps
    save_steps=10,  # Save checkpoint every X updates steps
    evaluation_strategy="steps",  # Evaluation is done (and logged) every X steps
    eval_steps=10,  # Evaluation every X steps
    warmup_steps=500,  # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,  # Strength of weight decay
    fp16=True,  # Enable fp16 for mixed precision training
    dataloader_pin_memory=False,  # Disable pinning memory for DataLoader
    report_to="none"  # Disable reporting to any integrations
)


# LoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=['k_proj', 'gate_proj', 'v_proj', 'up_proj', 'q_proj', 'o_proj', 'down_proj']
)


# Create a BitsAndBytesConfig object with the desired settings
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True  # Indicates that the model should be loaded in 4-bit quantization
    # Add any other configurations you need here
)

# Model to fine-tune with updated quantization configuration
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    quantization_config=bnb_config
)
model.config.use_cache = False

# Reference model with updated quantization configuration
ref_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    quantization_config=bnb_config
)



# Training arguments
training_args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    learning_rate=5e-5,
    lr_scheduler_type="cosine",
    max_steps=200,
    save_strategy="no",
    logging_steps=1,
    output_dir=new_model,
    optim="paged_adamw_32bit",
    warmup_steps=100,
    #bf16=True,
    fp16=True,  # Use fp16 precision instead of bf16
    report_to="wandb",
)



`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
# Create DPO trainer without a reference model
dpo_trainer = DPOTrainer(
    model=model,
    ref_model=None,  # Set ref_model to None when using peft_config
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,  # Your PEFT configuration
    beta=0.1,
    max_prompt_length=1024,
    max_length=1536,

)

# Fine-tune model with DPO
dpo_trainer.train()

/usr/local/lib/python3.10/dist-packages/trl/trainer/dpo_trainer.py:328: UserWarning: When using DPODataCollatorWithPadding, you should set `remove_unused_columns=False` in your TrainingArguments we have set it for you, but you should do it yourself in the future.
  warnings.warn(


Map:   0%|          | 0/12859 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:391: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn('Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:460: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True o

Step,Training Loss
1,0.693100
2,0.693100
3,0.691600
4,0.689700
5,0.688300
6,0.687900
7,0.678900
8,0.674200
9,0.659100
10,0.648800


KeyboardInterrupt: 

### Connect to Hugging Face for model upload
#### Logging to Hugging Face

To make sure the model can be uploaded to be used for Inference, it's necessary to log in to the Hugging Face hub.

#### Getting a Hugging Face token
Steps:
1. Navigate to this URL: https://huggingface.co/settings/tokens
2. Create a `write` token and copy it to your clipboard
3. Run the code below and enter your token



In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# upload th emodel

In [ ]:
# Save artifacts
dpo_trainer.model.save_pretrained("final_checkpoint")
tokenizer.save_pretrained("final_checkpoint")

# Flush memory
del dpo_trainer, model, ref_model
gc.collect()
torch.cuda.empty_cache()

# Reload model in FP16 (instead of NF4)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    return_dict=True,
    torch_dtype=torch.float16,
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Merge base model with the adapter
model = PeftModel.from_pretrained(base_model, "final_checkpoint")
model = model.merge_and_unload()

# Save model and tokenizer
model.save_pretrained(new_model)
tokenizer.save_pretrained(new_model)

# Push them to the HF Hub
model.push_to_hub(new_model, use_temp_dir=False, token=hf_token)
tokenizer.push_to_hub(new_model, use_temp_dir=False, token=hf_token)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Upload 3 LFS files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.18k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/tvkkishore/inspire-Mistral-7B-v2-DPO-Math/commit/e241d179aa9f07617e6198428c449fe783860380', commit_message='Upload tokenizer', commit_description='', oid='e241d179aa9f07617e6198428c449fe783860380', pr_url=None, pr_revision=None, pr_num=None)

@ inference

In [ ]:
# Corrected format for the prompt
message = [
    # {"role": "system", "content": "You are a helpful assistant chatbot."},  # This line should be removed or commented out
    {"role": "user", "content": "What is a Large Language Model?"},  # The conversation starts with the user's message
    {"role": "assistant", "content": "A Large Language Model (LLM) is..."}  # The assistant's response
]

# Ensure you have the correct model name for the tokenizer
tokenizer = AutoTokenizer.from_pretrained(new_model)

# Apply the chat template without the system role
prompt = tokenizer.apply_chat_template(message, add_generation_prompt=True, tokenize=False)

# Create pipeline
pipeline = transformers.pipeline(
    "text-generation",
    model=new_model,
    tokenizer=tokenizer
)

# Generate text
sequences = pipeline(
    prompt,
    do_sample=True,
    temperature=0.7,
    top_p=0.9,
    num_return_sequences=1,
    max_length=200,
)
print(sequences[0]['generated_text'])

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
A decoder-only architecture is being used, but right-padding was detected! For correct generation results, please set `padding_side='left'` when initializing the tokenizer.


<s>[INST] What is a Large Language Model? [/INST]A Large Language Model (LLM) is...</s>as a type of artificial intelligence (AI) system designed to understand and generate human-like text based on the input data it is given. The term "large" refers to the size of the model, specifically the number of parameters it has.

Large language models are trained on vast amounts of text data, using techniques such as deep learning and natural language processing. They learn to identify patterns and relationships in the data, enabling them to understand and generate text in a contextually appropriate manner. These models can be used for various applications, such as text completion, translation, summarization, and question answering. They have achieved impressive results in tasks that require understanding and generating human-like text, but they also come with challenges related to their size, cost, and potential risks, such as generating misinformation or invading privacy.


## Completed 🎉
After the command above is completed your Model will be uploaded to Hugging Face.

#### Learn more about AutoTrain (optional)
If you want to learn more about what command-line flags are available
